## Python 中的属性描述符和属性查找过程
- 这个属性描述符，有点像 JS 中的原型链 prototype

### 假设如下的 User 类，我如何将验证层加到 User 类的定义中呢

In [1]:
from datetime import date, datetime

class User:
    def __init__(self, name, email, birthday):
        self.name = name
        self.email = email
        self.birthday = birthday
        # 计算属性进行 set，添加 _age 隐藏属性
        self._age = 0
    def get_age(self):
        return datetime.now().year - self.birthday.year
    
    @property
    def age(self):
        return datetime.now().year - self.birthday.year

    @age.setter
    def age(self, value):
        self._age = value


if __name__ == "__main__":
    user =  User("bobby", date(year=1987, month=6, day=1))
    print("user.age = ", user.age)

user.age =  38


In [12]:
import numbers

# 实现 __get__，__set__，__delete__ 中的任意一个方法，即实现了属性描述符
class IntField(object):
    # 数据描述符（实现了 get 和 set 类似于 java 中的 @Data）
    def __get__(self, instance, owner):
        return self.value
    def __set__(self, instance, value):
        if not isinstance(value, numbers.Integral):
            raise ValueError("int value needed.")
        if value < 0:
            raise ValueError("positive value needed.")
        # 这里不能继续 instance.age，会导致递归调用
        self.value = value
    def __delete__(self, instance):
        pass

class NonDataIntField:
    # 非数据属性描述符（没有同时实现 get 和 set）
    def __get__(self, instance, owner):
        return self.value

"""
如果 model 是某个类的实例，那么 model.age
首先调用 __getattribute__。
如果类定义了 __getattr__ 方法，那么在 __getattribute__ 抛出 AttributeError 的时候就会调用到 __getattr__
而对于描述符(__get__) 的调用，则是发生在 __getattribute__ 内部的。
model =Model()，那么 model.age 顺序如下:

(1) 如果“age”是出现在 Model 或其基类（类型的静态变量）的 __dict__ 中，并且age是data descriptor（类似于 age=IntField()），那么调用其 __get__，否则

(2) 如果“age”出现在 obj（对象） 的 __dict__ 中，那么直接返回 obj.__dict__['age']，否则

(3) 如果“age”出现在 Model 或其基类的 __dict__ 中，并且 age 不是 data descriptor

(3.1) 如果 age 是 non-data descriptor，那么调用其 __get__ 方法，否则

(3.2) 返回 __dict__['age']

(4) 如果 Model 有 __getattr__ 方法，调用 __getattr__ 方法，否则

(5) 抛出 AttributeError
"""
class Model():
    age = IntField()

if __name__ == "__main__":
    model = Model()
    model.age = 30
    # age 没有进入到 user 的 __dict__ 中
    print("model.__dict__ = ", model.__dict__)
    print("model.age = ", model.age)
    print("getattr(model, 'age') = ", getattr(model, "age"))

model.__dict__ =  {}
model.age =  30
getattr(model, 'age') =  30


### age 进入到对象的 dict 中，因为 对象的 dict 优先级高于非数据描述符

In [13]:
import numbers

class NonDataIntField:
    # 非数据属性描述符（没有同时实现 get 和 set）
    def __get__(self, instance, owner):
        return self.value

class Model():
    age = NonDataIntField()

if __name__ == "__main__":
    model = Model()
    model.age = 30
    # age 没有进入到 model 的 __dict__ 中
    print("model.__dict__ = ", model.__dict__)
    print("model.age = ", model.age)

model.__dict__ =  {'age': 30}
model.age =  30
